In [ ]:
using ArgParse
using JSON

include("./src/Types.jl")
include("./src/Macros.jl")
include("./src/Parser.jl")
include("./src/Problem.jl")
include("./src/strategy/JuliaStrategy.jl")
include("./src/Common.jl")

In [2]:
# Grab the required functions for code generation -
# const parser_function::Function = parse_vff_file

function parse_commandline()
    settings_object = ArgParseSettings()
    @add_arg_table settings_object begin
      "-o"
        help = "Directory where the Matlab model files will be written."
        arg_type = AbstractString
        default = "."

      "-m"
        help = "Path to the biochemical reaction file written in the vff format."
        arg_type = AbstractString
        required = true

        "-s"
        help = "Host type: bacteria or mammalian?"
        arg_type = Symbol
        default = :bacteria
    end

    # return a dictionary w/args -
    return parse_args(settings_object)
end

parse_commandline (generic function with 1 method)

In [64]:
function vff_metabolic_sentence_factory(sentence::String,handler_symbol::Symbol)

    # Initialize -
    sentence_vector = VFFSentence[]

    # Ok, so now we have the array for sentences -
    vff_sentence = VFFSentence()
    vff_sentence.original_sentence = sentence

    # split the sentence -
    split_array = split(sentence,",")
    vff_sentence.sentence_name = split_array[1]
    #vff_sentence.sentence_type_flag = parse(Int,convert(String,split_array[2]))
    vff_sentence.catalyst_lexeme = "[]"#convert(String,split_array[3])
    vff_sentence.sentence_reactant_clause = split_array[2]
    vff_sentence.sentence_product_clause = split_array[3]
    vff_sentence.sentence_reverse_bound = parse(Float64,split_array[4])
    vff_sentence.sentence_forward_bound = parse(Float64,split_array[5])
    vff_sentence.sentence_handler = handler_symbol
    vff_sentence.sentence_delimiter = ','
    vff_sentence.sentence_ec_number = "[]"#split_array[6]

    # add sentence to sentence_vector -
    push!(sentence_vector,vff_sentence)

    # Check - is this reversible?
    if (vff_sentence.sentence_reverse_bound == -Inf)

        # ok, so we have a reversible reaction -
        # first change lower bound to 0 -
        vff_sentence.sentence_reverse_bound = 0.0

        # create a new copy of sentence -
        vff_sentence_copy = deepcopy(vff_sentence)
        vff_sentence_copy.sentence_name = (vff_sentence_copy.sentence_name)*"_reverse"
        vff_sentence_copy.sentence_reactant_clause = vff_sentence.sentence_product_clause
        vff_sentence_copy.sentence_product_clause = vff_sentence.sentence_reactant_clause
        vff_sentence_copy.sentence_handler = vff_sentence.sentence_handler
        vff_sentence_copy.sentence_type_flag = vff_sentence.sentence_type_flag
        vff_sentence_copy.sentence_ec_number = vff_sentence.sentence_ec_number

        # add sentence and sentence copy to the expanded_sentence_vector -
        push!(sentence_vector,vff_sentence_copy)
    end

    return sentence_vector
end


vff_metabolic_sentence_factory (generic function with 1 method)

In [65]:
function parse_vff_metabolic_statements(path_to_model_file::AbstractString)

  # We are going to load the sentences in the file into a vector
  # if not a valid model file, then throw an error -
  sentence_vector = VFFSentence[]
  expanded_sentence_vector::Array{VFFSentence} = VFFSentence[]
  tmp_array::Array{AbstractString} = AbstractString[]
  desired_handler_symbol::Symbol = :metabolism_handler
  current_handler_symbol::Symbol = :metabolism_handler

  try

    # Open the model file, and read each line into a vector -
    open(path_to_model_file,"r") do model_file
      for line in eachline(model_file)

          if (contains(line,"//") == false && search(line,"\n") == 0:-1 && line != "")
            push!(tmp_array,chomp(line))
          end
      end
    end

    @show tmp_array

    for sentence in tmp_array

      # convert substring to string -
      local_sentence = convert(String,sentence)
        # Check the current pragma -
        if (contains(local_sentence,"#pragma") == true)

            # Extract the current handler -
            current_handler_symbol = extract_vff_handler_symbol(local_sentence)
            println("Handler: ", current_handler_symbol)
        end

        # Depending upon the handler_symbol, we will process diff types of records -
        if (current_handler_symbol == desired_handler_symbol && contains(local_sentence,"#pragma") == false)
            local_vff_sentence_array = vff_metabolic_sentence_factory(local_sentence,current_handler_symbol)
            for local_vff_sentence in collect(local_vff_sentence_array)
              push!(expanded_sentence_vector,local_vff_sentence)
            end
        elseif (current_handler_symbol == :transcription_handler && contains(local_sentence,"#pragma") == false)

            local_vff_sentence_array = vff_transcription_sentence_factory(local_sentence,current_handler_symbol)
            for local_vff_sentence in collect(local_vff_sentence_array)
              push!(expanded_sentence_vector,local_vff_sentence)
            end

        elseif (current_handler_symbol == :translation_handler && contains(local_sentence,"#pragma") == false)

            local_vff_sentence_array = vff_translation_sentence_factory(local_sentence,current_handler_symbol)
            for local_vff_sentence in collect(local_vff_sentence_array)
              push!(expanded_sentence_vector,local_vff_sentence)
            end

        elseif (current_handler_symbol == :degradation_handler && contains(local_sentence,"#pragma") == false)

            local_vff_sentence_array = vff_degradation_sentence_factory(local_sentence,current_handler_symbol)
            for local_vff_sentence in collect(local_vff_sentence_array)
              push!(expanded_sentence_vector,local_vff_sentence)
            end

        elseif (current_handler_symbol == :infrastructure_handler && contains(local_sentence,"#pragma") == false)

            local_vff_sentence_array = vff_infrastructure_sentence_factory(local_sentence,current_handler_symbol)
            for local_vff_sentence in collect(local_vff_sentence_array)
              push!(expanded_sentence_vector,local_vff_sentence)
            end
        end
    end

  catch err
    showerror(STDOUT, err, backtrace());println()
  end

  return expanded_sentence_vector
end

parse_vff_metabolic_statements (generic function with 1 method)

In [69]:
function main()

  # Build the arguement dictionary -
    #"../Sequence-Specific-FBA-CFPS-Publication-Code/test/Reactions.net",
  parsed_args = Dict("m" => "../Sequence-Specific-FBA-CFPS-Publication-Code/test/Reactions.net",
                 "s" => :bacteria,
                 "o" => "../Sequence-Specific-FBA-CFPS-Publication-Code/BTOL_Simp/")
    #parse_commandline()

  # Load the statement_vector -
  path_to_model_file = parsed_args["m"]
  metabolic_statement_vector::Array{VFFSentence} = parse_vff_metabolic_statements(path_to_model_file)
  println(metabolic_statement_vector)

  # Load the JSON configuration file -
  config_dict = JSON.parsefile("./config/Configuration.json")
    
  # Load the enzyme kinetics JSON file -
  enzyme_kinetics_dictionary = JSON.parsefile("./config/Enzymes.json")

  # Generate the problem object -
  problem_object = generate_problem_object(metabolic_statement_vector,config_dict)
  problem_object.configuration_dictionary = config_dict
  problem_object.enzyme_kinetics_dictionary = enzyme_kinetics_dictionary

  # Write the DataDictionary -
  component_set = Set{ProgramComponent}()

  # write debug buffer -
  program_component_debug = build_debug_buffer(problem_object)
  push!(component_set,program_component_debug)
  progress_message = "constructed the debug buffer ..."
  println(progress_message)

  # data dictionary -
  host_type = parsed_args["s"]
  program_component_data_dictionary = build_data_dictionary_buffer(problem_object,host_type)
  push!(component_set,program_component_data_dictionary)
  progress_message = "constructed the data_dictionary buffer ..."
  println(progress_message)

  # Write the stoichiometric_matrix --
  program_component_stoichiometric_matrix = generate_stoichiomteric_matrix_buffer(problem_object)
  push!(component_set,program_component_stoichiometric_matrix)

  # Dump the component_set to disk -
  path_to_output_file = parsed_args["o"]
  write_program_components_to_disk(path_to_output_file,component_set)

  # Transfer distrubtion files to the output -
  transfer_distribution_file("./distribution","FluxDriver.jl",path_to_output_file,"FluxDriver.jl")
  transfer_distribution_file("./distribution","Solve.jl",path_to_output_file,"Solve.jl")
  transfer_distribution_file("./distribution","DSolve.jl",path_to_output_file,"DSolve.jl")
  transfer_distribution_file("./distribution","Include.jl",path_to_output_file,"Include.jl")
  transfer_distribution_file("./distribution","Utility.jl",path_to_output_file,"Utility.jl")
  transfer_distribution_file("./distribution","README_INCLUDE.md",path_to_output_file,"README.md")
  transfer_distribution_file("./distribution","Atom.csv",path_to_output_file,"Atom.txt")

end

main()

tmp_array = AbstractString["#pragma::metabolism_handler", "R_glk_atp,M_atp_c+M_glc_D_c,M_adp_c+M_g6p_c,0,inf", "R_pgi,M_g6p_c,M_f6p_c,-inf,inf", "R_pfk,M_atp_c+M_f6p_c,M_adp_c+M_fdp_c,0,inf", "R_fdp,M_fdp_c+M_h2o_c,M_f6p_c+M_pi_c,0,inf", "R_fbaA,M_fdp_c,M_dhap_c+M_g3p_c,-inf,inf", "R_tpiA,M_dhap_c,M_g3p_c,-inf,inf", "R_gapA,M_g3p_c+M_nad_c+M_pi_c,M_13dpg_c+M_h_c+M_nadh_c,-inf,inf", "R_pgk,M_13dpg_c+M_adp_c,M_3pg_c+M_atp_c,-inf,inf", "R_gpm,M_3pg_c,M_2pg_c,-inf,inf", "R_eno,M_2pg_c,M_h2o_c+M_pep_c,-inf,inf", "R_pyk,M_adp_c+M_pep_c,M_atp_c+M_pyr_c,0,inf", "R_pck,M_atp_c+M_oaa_c,M_adp_c+M_co2_c+M_pep_c,0,inf", "R_ppc,M_co2_c+M_h2o_c+M_pep_c,M_oaa_c+M_pi_c,0,inf", "R_pdh,M_coa_c+M_nad_c+M_pyr_c,M_accoa_c+M_co2_c+M_nadh_c+M_h_c,0,inf", "R_pps,M_atp_c+M_h2o_c+M_pyr_c,M_amp_c+M_pep_c+M_pi_c,0,inf", "R_zwf,M_g6p_c+M_nadp_c,M_6pgl_c+M_h_c+M_nadph_c,-inf,inf", "R_pgl,M_6pgl_c+M_h2o_c,M_6pgc_c,0,inf", "R_gnd,M_6pgc_c+M_nadp_c,M_co2_c+M_nadph_c+M_ru5p_D_c+M_h_c,0,inf", "R_rpe,M_ru5p_D_c,M_xu5p_D_c


Handler: metabolism_handler


VFFSentence[VFFSentence("R_glk_atp,M_atp_c+M_glc_D_c,M_adp_c+M_g6p_c,0,inf", "R_glk_atp", 102404, "M_atp_c+M_glc_D_c", "M_adp_c+M_g6p_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_pgi,M_g6p_c,M_f6p_c,-inf,inf", "R_pgi", 102404, "M_g6p_c", "M_f6p_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_pgi,M_g6p_c,M_f6p_c,-inf,inf", "R_pgi_reverse", 102404, "M_f6p_c", "M_g6p_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_pfk,M_atp_c+M_f6p_c,M_adp_c+M_fdp_c,0,inf", "R_pfk", 102404, "M_atp_c+M_f6p_c", "M_adp_c+M_fdp_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_fdp,M_fdp_c+M_h2o_c,M_f6p_c+M_pi_c,0,inf", "R_fdp", 102404, "M_fdp_c+M_h2o_c", "M_f6p_c+M_pi_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_fbaA,M_fdp_c,M_dhap_c+M_g3p_c,-inf,inf", "R_fbaA", 102404, "M_fdp_c", "M_dhap_c+M_g3p_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_fbaA,M_fdp_c,M_dhap_c+M_g3p_c,-inf,inf", "R_

"[]", "[]"), VFFSentence("R_mglx_deg,M_mglx_c+M_nad_c+M_h2o_c,M_pyr_c+M_nadh_c+M_h_c,0,inf", "R_mglx_deg", 102404, "M_mglx_c+M_nad_c+M_h2o_c", "M_pyr_c+M_nadh_c+M_h_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_ser_deg,M_ser_L_c,M_nh3_c+M_pyr_c,0,inf", "R_ser_deg", 102404, "M_ser_L_c", "M_nh3_c+M_pyr_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_pro_deg,M_pro_L_c+M_q8_c+2.0*M_h2o_c+M_nad_c,M_h_c+M_q8h2_c+M_nadh_c+M_glu_L_c,0,inf", "R_pro_deg", 102404, "M_pro_L_c+M_q8_c+2.0*M_h2o_c+M_nad_c", "M_h_c+M_q8h2_c+M_nadh_c+M_glu_L_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_thr_deg1,M_thr_L_c+M_nad_c+M_coa_c,M_nadh_c+M_h_c+M_accoa_c+M_gly_L_c,0,inf", "R_thr_deg1", 102404, "M_thr_L_c+M_nad_c+M_coa_c", "M_nadh_c+M_h_c+M_accoa_c+M_gly_L_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_thr_deg2,M_thr_L_c+M_nad_c+M_o2_c+M_h2o_c,M_nadh_c+M_co2_c+M_h2o2_c+M_nh3_c+M_mglx_c+M_h_c,0,inf", "R_thr_deg2", 102404, "M_thr_

_c,0,inf", "R_ctp_cmp", 102404, "M_ctp_c+M_h2o_c", "M_cmp_c+M_ppi_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_gtp_gmp,M_gtp_c+M_h2o_c,M_gmp_c+M_ppi_c,0,inf", "R_gtp_gmp", 102404, "M_gtp_c+M_h2o_c", "M_gmp_c+M_ppi_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_atp_adp,M_atp_c+M_h2o_c,M_adp_c+M_pi_c,0,inf", "R_atp_adp", 102404, "M_atp_c+M_h2o_c", "M_adp_c+M_pi_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_utp_adp,M_utp_c+M_h2o_c,M_udp_c+M_pi_c,0,inf", "R_utp_adp", 102404, "M_utp_c+M_h2o_c", "M_udp_c+M_pi_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_ctp_adp,M_ctp_c+M_h2o_c,M_cdp_c+M_pi_c,0,inf", "R_ctp_adp", 102404, "M_ctp_c+M_h2o_c", "M_cdp_c+M_pi_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_gtp_adp,M_gtp_c+M_h2o_c,M_gdp_c+M_pi_c,0,inf", "R_gtp_adp", 102404, "M_gtp_c+M_h2o_c", "M_gdp_c+M_pi_c", 0.0, Inf, ',', :metabolism_handler, "[]", "[]"), VFFSentence("R_udp_utp,M_udp_c+M_a

...

Processing R_or_syn_1 index 133 of 277
Processing R_or_syn_2 index 134 of 277
Processing R_omp_syn index 135 of 277
Processing R_ump_syn index 136 of 277
Processing R_ctp_1 index 137 of 277
Processing R_ctp_2 index 138 of 277
Processing R_A_syn_1 index 139 of 277
Processing R_A_syn_2 index 140 of 277
Processing R_A_syn_3 index 141 of 277
Processing R_A_syn_4 index 142 of 277
Processing R_A_syn_5 index 143 of 277
Processing R_A_syn_6 index 144 of 277
Processing R_A_syn_7 index 145 of 277
Processing R_A_syn_8 index 146 of 277
Processing R_A_syn_9 index 147 of 277
Processing R_A_syn_10 index 148 of 277
Processing R_A_syn_12 index 149 of 277
Processing R_xmp_syn index 150 of 277
Processing R_gmp_syn index 151 of 277
Processing R_atp_amp index 152 of 277
Processing R_utp_ump index 153 of 277
Processing R_ctp_cmp index 154 of 277
Processing R_gtp_gmp index 155 of 277
Processing R_atp_adp index 156 of 277
Processing R_utp_adp index 157 of 277
Processing R_ctp_adp index 158 of 277
Processing R

Processing R_pro index 94 of 277
Processing R_serABC index 95 of 277
Processing R_thr index 96 of 277
Processing R_trp index 97 of 277
Processing R_tyr index 98 of 277
Processing R_val index 99 of 277
Processing R_arg_deg index 100 of 277
Processing R_asp_deg index 101 of 277
Processing R_asn_deg index 102 of 277
Processing R_gly_deg index 103 of 277
Processing R_mglx_deg index 104 of 277
Processing R_ser_deg index 105 of 277
Processing R_pro_deg index 106 of 277
Processing R_thr_deg1 index 107 of 277
Processing R_thr_deg2 index 108 of 277
Processing R_thr_deg3 index 109 of 277
Processing R_trp_deg index 110 of 277
Processing R_cys_deg index 111 of 277
Processing R_lys_deg index 112 of 277
Processing R_gln_deg index 113 of 277
Processing R_glu_deg index 114 of 277
Processing R_gaba_deg1 index 115 of 277
Processing R_gaba_deg2 index 116 of 277
Processing R_chor index 117 of 277
Processing R_fol_e index 118 of 277
Processing R_fol_1 index 119 of 277
Processing R_fol_2a index 120 of 277
P

Processing species symbol M_asn_L_c_tRNA_c (index 29 of 145)
Processing species symbol M_asp_L_c (index 30 of 145)
Processing species symbol M_asp_L_c_tRNA_c (index 31 of 145)
Processing species symbol M_atp_c (index 32 of 145)
Processing species symbol M_b2coa_c (index 33 of 145)
Processing species symbol M_btal_c (index 34 of 145)
Processing species symbol M_btcoa_c (index 35 of 145)
Processing species symbol M_btol_c (index 36 of 145)
Processing species symbol M_cadav_c (index 37 of 145)
Processing species symbol M_cair_c (index 38 of 145)
Processing species symbol M_cdp_c (index 39 of 145)
Processing species symbol M_chor_c (index 40 of 145)
Processing species symbol M_cit_c (index 41 of 145)
Processing species symbol M_clasp_c (index 42 of 145)
Processing species symbol M_cmp_c (index 43 of 145)
Processing species symbol M_co2_c (index 44 of 145)
Processing species symbol M_coa_c (index 45 of 145)
Processing species symbol M_ctp_c (index 46 of 145)
Processing species symbol M_cys_

Processing M_nh3_c index 106 of 145
Processing M_o2_c index 107 of 145
Processing M_oaa_c index 108 of 145
Processing M_omp_c index 109 of 145
Processing M_or_c index 110 of 145
Processing M_pep_c index 111 of 145
Processing M_phe_L_c index 112 of 145
Processing M_phe_L_c_tRNA_c index 113 of 145
Processing M_pi_c index 114 of 145
Processing M_ppi_c index 115 of 145
Processing M_pro_L_c index 116 of 145
Processing M_pro_L_c_tRNA_c index 117 of 145
Processing M_prop_c index 118 of 145
Processing M_prpp_c index 119 of 145
Processing M_pyr_c index 120 of 145
Processing M_q8_c index 121 of 145
Processing M_q8h2_c index 122 of 145
Processing M_r5p_c index 123 of 145
Processing M_ru5p_D_c index 124 of 145
Processing M_s7p_c index 125 of 145
Processing M_saicar_c index 126 of 145
Processing M_ser_L_c index 127 of 145
Processing M_ser_L_c_tRNA_c index 128 of 145
Processing M_succ_c index 129 of 145
Processing M_succoa_c index 130 of 145
Processing M_thf_c index 131 of 145
Processing M_thr_L_c i

STM - processing species M_tyr_L_c_tRNA_c (137 of 145) ... 
STM - processing species M_udp_c (138 of 145) ... 
STM - processing species M_ump_c (139 of 145) ... 
STM - processing species M_utp_c (140 of 145) ... 
STM - processing species M_val_L_c (141 of 145) ... 
STM - processing species M_val_L_c_tRNA_c (142 of 145) ... 
STM - processing species M_xmp_c (143 of 145) ... 
STM - processing species M_xu5p_D_c (144 of 145) ... 
STM - processing species tRNA_c (145 of 145) ... 
